In [1]:
import pandas as pd
import os
from glob import glob
import reports
pd.options.display.float_format = '{:.2f}'.format

In [7]:
mlm_ddt_filepaths_format = '/media/dmlab/My Passport/DATA/cross-domain/finetune_{}/source=*_post=MLM+DDT_target=*/classification_report.csv'
kfold_num = 1

In [10]:
def get_average_classification_report(reports):
    for i, report in enumerate(reports):
        report.set_index(report.columns[0], inplace=True)
        if i == 0: 
            report_avg = report
        else: 
            report_avg = report_avg.add(report)
    report_avg = report_avg / len(reports)
    return report_avg

def show_result(filepaths_format, kfold_num=1):
    kfold_reports = []
    for i in range(0, kfold_num):
        filepaths = glob(filepaths_format.format(i))

        records = []
        for filepath in filepaths:
            source, post, target = os.path.basename(os.path.dirname(filepath)).split('_')
            source, target = source.replace('source=',''), target.replace('target=', '')

            df = pd.read_csv(filepath)
            df.set_index('Unnamed: 0', inplace=True)
            acc = 100*df.filter(items = ['accuracy'], axis=0)['f1-score'].values[0]

            records.append(('{}->{}'.format(source, target), float(acc)))

        result_df = pd.DataFrame(records, columns = ['source->target', '{}_accuracy'])
        kfold_reports.append(result_df)

    result_df = get_average_classification_report(kfold_reports)
    result_df.loc['Average'] = result_df.mean()
    return result_df

* Post-training tasks: (1) MLM (2) DDT
* 파인튜닝: Raw 소스 (train), Raw 타겟 (test)
* No k-fold

In [11]:
pt_mlm_ddt = show_result(mlm_ddt_filepaths_format, kfold_num=kfold_num)
pt_mlm_ddt.columns = ['MLM+DDT']

result = pd.concat([pt_mlm_ddt], axis=1)
result

,MLM+DDT
source->target,
books->dvd,87.25
books->electronics,72.90
books->kitchen,64.00
dvd->books,83.10
dvd->electronics,77.60
dvd->kitchen,53.15
electronics->books,53.60
electronics->dvd,56.20
electronics->kitchen,88.70
